In [1]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver.firefox.options import Options
import time

In [2]:
options = Options()
options.add_argument("--disable-notifications")
options.add_argument('start-maximized')

In [6]:
#chrome = webdriver.Chrome('./chromedriver', chrome_options=options)
chrome = webdriver.Firefox(executable_path=r'/Users/mingtsung/Desktop/爬蟲/geckodriver', options=options)
obj_link = []
for i in range(2):
    chrome.get("https://www.amazon.com/Best-Sellers-Clothing-Shoes-Jewelry-Womens-Cocktail-Dresses/zgbs/fashion/11006703011/ref=zg_bs_nav_fashion_4_2346727011/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}".format(i+1, i+1))
    chrome.implicitly_wait(2)
    js = "var q=document.documentElement.scrollTop=5500"
    chrome.execute_script(js)
    time.sleep(1)
    js = "var q=document.documentElement.scrollTop=7800"
    chrome.execute_script(js)
    time.sleep(1)
    cats=chrome.find_elements(By.XPATH, "//div[@class = 'zg-grid-general-faceout']/div/a[1]")
    for cat in cats:
        url = cat.get_attribute('href')
        obj_link.append(url)
    print(len(obj_link))


<ipython-input-6-2450151217bd>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Firefox(executable_path=r'/Users/mingtsung/Desktop/爬蟲/geckodriver', options=options)


50
100


In [28]:
chrome = webdriver.Firefox(executable_path=r'/Users/mingtsung/Desktop/爬蟲/geckodriver', options=options)

<ipython-input-28-10af23f0ead0>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Firefox(executable_path=r'/Users/mingtsung/Desktop/爬蟲/geckodriver', options=options)


In [29]:
name, price, seller, rating, top_reviews, rating_count, question_count = [], [], [], [], [], [], []
for j in obj_link:
    chrome.get(j)
    n = chrome.find_elements(By.XPATH, "//div[@id = 'titleSection']/h1/span[1]")
    n = n[0].get_attribute('textContent')
    name.append(str.strip(n))
    try:
        p = chrome.find_elements(By.XPATH, "//div[@class = 'a-section a-spacing-none aok-align-center']/span/span[1]")
        p = p[0].get_attribute('textContent')
        price.append(p[1:])
    except:
        p = chrome.find_elements(By.XPATH, "//span[@class = 'a-offscreen']")
        p = p[0].get_attribute('textContent')
        price.append(p[1:])
    try:
        s = chrome.find_elements(By.XPATH, "//div[@class = 'a-section a-spacing-none']/a[1]")
        s = s[0].get_attribute('textContent')
        if 'out' in s:
            s = chrome.find_elements(By.XPATH, "//div[@id = 'amznStoresBylineLogoTextContainer']/a[1]")
            s = s[0].get_attribute('textContent')
            seller.append(str.strip(s))
        else:
            seller.append(s)
    except:
        s = chrome.find_elements(By.XPATH, "//div[@id = 'amznStoresBylineLogoTextContainer']/a[1]")
        s = s[0].get_attribute('textContent')
        seller.append(str.strip(s))
    r = chrome.find_elements(By.XPATH, "//span[@class = 'a-size-medium a-color-base']")
    r = r[0].get_attribute('textContent')
    rating.append(r)
    re = chrome.find_elements(By.XPATH, "//div[@class = 'a-section review aok-relative']//div[@class = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content']/span")
    reviews = []
    for k in re:
        review = k.get_attribute('textContent')
        reviews.append(review)
    top_reviews.append(reviews)
    
    rc = chrome.find_elements(By.XPATH, "//span[@id = 'acrCustomerReviewText']")
    rc = rc[0].get_attribute('textContent')[:-8]
    rating_count.append(str.strip(rc))
    
    try:
        q = chrome.find_elements(By.XPATH, "//a[@class = 'a-link-normal askATFLink']/span")
        q = q[0].get_attribute('textContent')[:-20]
        question_count.append(str.strip(q))
    except:
        question_count.append(0)
chrome.close()

In [30]:
df_amazon = pd.DataFrame(list(zip(name, price, seller, rating, top_reviews, rating_count, question_count)), columns = ['name', 'price', 'seller','rating', 'top_reviews', 'rating_count', 'question_count'])
df_amazon.to_csv("amazon_dresses2.csv",index=False)